In [ ]:
import torch, torchvision, torchaudio
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import os, pickle, math, random
from copy import deepcopy
device ='cuda' if torch.cuda.is_available() else 'cpu'
torch.__version__, torchvision.__version__

('2.5.1+cu124', '0.20.1+cu124')

#### Multi-Category Classification

In [ ]:
''' multi-category classification '''
torch.manual_seed(42)
transform=T.Compose(
    [T.Resize((224, 224)),
     T.ToTensor(),
     T.Normalize(mean=[0.485,0.456, 0.406], std = [0.229, 0.224, 0.225])
])
train_set = torchvision.datasets.Country211(
    root = '.', split='train', download=True, transform=transform ) #192
test_set = torchvision.datasets.Country211(
    root = '.', split='test', download=True, transform=transform ) #50
validation_set = test_set = torchvision.datasets.Country211(
    root = '.', split='valid', download=True, transform=transform ) #50

batch_size = 64
dataloader_train = DataLoader(train_set, batch_size, shuffle=True)
dataloader_test = DataLoader(test_set, batch_size, shuffle=False)
dataloader_validation = DataLoader(validation_set, batch_size, shuffle=False)


In [ ]:
class EarlyStop:
  def __init__(self, patience=10):
    self.patience, self.steps, self.min_loss = patience, 0, float('inf')

  def stop(self, validation_loss):
    if validation_loss < self.min_loss:
      self.min_loss, self.steps = validation_loss, 0
    elif validation_loss >= self.min_loss:
      self.steps +=1
    if self.steps >= self.patience: return True
    else: return False

stopper = EarlyStop()

In [ ]:
model_ = nn.Sequential(
  nn.Linear(3*224*224, 256),
  nn.ReLU(),
  nn.Linear(256, 128),
  nn.ReLU(),
  nn.Linear(128, 64),
  nn.ReLU(),
  nn.Linear(64, 10)).to(device)

lr = 0.001
optimizer = torch.optim.Adam(model_.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
''' train the multi-category classifier '''
def train_epoch():
  training_loss = 0
  for n, (images, labels) in enumerate(dataloader_train):
    images = images.reshape(-1, 3*224*224).to(device)
    labels = labels.reshape(-1,).to(device)
    predictions = model_(images)
    loss = loss_fn(predictions, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    training_loss += loss
  return training_loss/n

def validation_epoch():
  validation_loss = 0
  for n, (images, labels) in enumerate(dataloader_validation):
    images = images.reshape(-1, 3*224*224).to(device)
    labels = labels.reshape(-1, ).to(device)
    predictions = model_(images)
    loss = loss_fn(predictions, labels)
    validation_loss +=loss
  return validation_loss/n

In [ ]:
for i in range(1, 10):
  training_loss = train_epoch()
  validation_loss = validation_epoch()
  print(f'at epoch {i}, \
        training loss {training_loss}, validation loss {validation_loss}')
  if stopper.stop(validation_loss) == True:
    break

''' accuracy of multi category classifier '''
results = []
for images, labels in dataloader_test:
  images = images.reshape(-1, 3*224*224).to(device)
  labels = (labels).reshape(-1,).to(device)
  predictions = model_(images)
  predictions_ = torch.argmax(predictions, dim=1)
  correct = (predictions_ == labels)
  results.append(correct.detach().cpu().numpy().mean())

accuracy = np.array(results).mean()
accuracy

##### CrossEntropyLoss combining LogSoftmax and NLLoss instead of  CrossEntropyLoss and SoftMax activation

In [ ]:
lr = 0.001
optimizer = torch.optim.Adam(model_.parameters(), lr=lr)
loss_fn = nn.NLLLoss()

for i in range(1, 10):
  training_loss = train_epoch()
  validation_loss = validation_epoch()
  print(f'at epoch {i}, training loss {training_loss}, validation loss {validation_loss}')
  if stopper.stop(validation_loss) == True:
    break

''' accuracy of multi category classifier '''
results = []
for images, labels in dataloader_test:
  images = images.reshape(-1, 3*224*224).to(device)
  labels = (labels).reshape(-1,).to(device)
  predictions = model_(images)
  predictions_ = torch.argmax(predictions, dim=1)
  correct = (predictions_ == labels)
  results.append(correct.detach().cpu().numpy().mean())

accuracy = np.array(results).mean()
accuracy